#### ![Spark Logo](http://curriculum-release.s3-website-us-west-2.amazonaws.com/wiki-book/general/logo_spark.png)

## Time series Analysis of APPLE stock  with Apache Spark (Python)

d ## Importing libraries

In [4]:
from pyspark.sql import SQLContext,Window
from pyspark.sql import SQLContext, Window
from pyspark.sql.functions import* 
from pyspark.ml.regression import LinearRegression 
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from  pyspark.sql.functions import abs, sqrt
import pyspark.sql.functions as sf
import pyspark.sql.functions as Column

import pandas as pd
import datetime
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as sf
from pyspark.sql.window import Window
import numpy as np

from math import log
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
import matplotlib.pyplot as plt
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import SQLContext, Window 
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression 
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler

from pyspark.sql.functions import abs, sqrt
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType

d ## Reading data

In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.sql("SELECT * FROM aapl_csv WHERE YEAR(aapl_csv.date) BETWEEN 1980 AND 2018")
df.show(10)

+-------------------+--------+--------+--------+--------+---------+----------+
 Date| Open| High| Low| Close|Adj Close| Volume|
+-------------------+--------+--------+--------+--------+---------+----------+
1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.022919|1.172584E8|
1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021723| 4.39712E7|
1980-12-16 00:00:00|0.453125|0.453125|0.450893|0.450893| 0.020129| 2.6432E7|
1980-12-17 00:00:00|0.462054|0.464286|0.462054|0.462054| 0.020627| 2.16104E7|
1980-12-18 00:00:00|0.475446|0.477679|0.475446|0.475446| 0.021225| 1.83624E7|
1980-12-19 00:00:00|0.504464|0.506696|0.504464|0.504464| 0.02252| 1.21576E7|
1980-12-22 00:00:00|0.529018| 0.53125|0.529018|0.529018| 0.023616| 9340800.0|
1980-12-23 00:00:00|0.551339|0.553571|0.551339|0.551339| 0.024613| 1.17376E7|
1980-12-24 00:00:00|0.580357|0.582589|0.580357|0.580357| 0.025908| 1.20008E7|
1980-12-26 00:00:00|0.633929|0.636161|0.633929|0.633929| 0.0283| 1.38936E7|
+-------------------+--------+--------+--------+--------+---------+----------+
only showing top 10 rows

d ## Normalize data (Adj close) with log

In [8]:
df = df.withColumn("ticker", sf.lit("AAPL"))
df = df.withColumn("Log_Adj_Close", sf.log("Adj close"))
df = df.withColumn("log_open", sf.log("Open"))
df = df.withColumn("log_high", sf.log("High"))
df = df.withColumn("log_low", sf.log("Low"))
df = df.withColumn("log_Volume", sf.log("Volume"))
df.show(10)

+-------------------+--------+--------+--------+--------+---------+----------+------+-------------------+-------------------+--------------------+-------------------+------------------+
 Date| Open| High| Low| Close|Adj Close| Volume|ticker| Log_Adj_Close| log_open| log_high| log_low| log_Volume|
+-------------------+--------+--------+--------+--------+---------+----------+------+-------------------+-------------------+--------------------+-------------------+------------------+
1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.022919|1.172584E8| AAPL| -3.77578905322425|-0.6667136747072164| -0.6623755218931916|-0.6667136747072164| 18.57989060451556|
1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021723| 4.39712E7| AAPL| -3.829383656591175|-0.7157220861254052| -0.7157220861254052|-0.7202984933502578|17.599045432119592|
1980-12-16 00:00:00|0.453125|0.453125|0.450893|0.450893| 0.020129| 2.6432E7| AAPL| -3.905593679506084|-0.7915872533731978| -0.7915872533731978|-0.7965251856480028|17.090085955302843|
1980-12-17 00:00:00|0.462054|0.464286|0.462054|0.462054| 0.020627| 2.16104E7| AAPL| -3.881154409020783|-0.7720734808686366| -0.7672545383275174|-0.7720734808686366| 16.88868523826086|
1980-12-18 00:00:00|0.475446|0.477679|0.475446|0.475446| 0.021225| 1.83624E7| AAPL|-3.8525755561922908|-0.7435019979935891| -0.7388162904577791|-0.7435019979935891| 16.72581565356789|
1980-12-19 00:00:00|0.504464|0.506696|0.504464|0.504464| 0.02252| 1.21576E7| AAPL|-3.7933514706024343|-0.6842588576729776| -0.6798440887853203|-0.6842588576729776|16.313465046601532|
1980-12-22 00:00:00|0.529018| 0.53125|0.529018|0.529018| 0.023616| 9340800.0| AAPL| -3.745830866671124|-0.6367328498433473| -0.6325225587435105|-0.6367328498433473| 16.04990245964193|
1980-12-23 00:00:00|0.551339|0.553571|0.551339|0.551339| 0.024613| 1.17376E7| AAPL| -3.704480501957244|-0.5954054672195551| -0.5913652861062011|-0.5954054672195551|16.278307922164174|
1980-12-24 00:00:00|0.580357|0.582589|0.580357|0.580357| 0.025908| 1.20008E7| AAPL|-3.6532034480477957| -0.54411182147817| -0.5402733658732763| -0.54411182147817|16.300483872196818|
1980-12-26 00:00:00|0.633929|0.636161|0.633929|0.633929| 0.0283| 1.38936E7| AAPL|-3.5648934618012267|-0.4558182957883265|-0.45230355690505786|-0.4558182957883265| 16.44693886041504|
+-------------------+--------+--------+--------+--------+---------+----------+------+-------------------+-------------------+--------------------+-------------------+------------------+
only showing top 10 rows

d ## Prediction

In [10]:
w = Window().partitionBy("ticker").orderBy("date")
#Predictions with Adj close
df=df.withColumn("pdaily", sf.lag("Log_Adj_Close", 1).over(w))
df = df.withColumn("pweekly", sf.lag("Log_Adj_Close", 5).over(w))
df = df.withColumn("pbiweekly", sf.lag("Log_Adj_Close", 10).over(w))
df = df.withColumn("pmonthly", sf.lag("Log_Adj_Close", 21).over(w))
df = df.withColumn("pquarterly", sf.lag("Log_Adj_Close", 84).over(w))


d ## Return

In [12]:
#Returns
df = df.withColumn("log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 1).over(w)) / sf.lag("Log_Adj_Close", 5).over(w))
df = df.withColumn("weekly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 5).over(w)) / sf.lag("Log_Adj_Close", 10).over(w))
df = df.withColumn("biweekly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 10).over(w)) / sf.lag("Log_Adj_Close", 21).over(w))
df = df.withColumn("monthly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 21).over(w)) / sf.lag("Log_Adj_Close", 42).over(w))
df = df.withColumn("quarterly_log_return", -1* (sf.col("Log_Adj_Close") - sf.lag("Log_Adj_Close", 84).over(w)) / sf.lag("Log_Adj_Close", 84).over(w))


d ## Volume

In [14]:
#Volume
df = df.withColumn("daily_volume_diff", (sf.col("log_Volume") - sf.lag("log_Volume", 1).over(w))) 
df = df.withColumn("weekly_volume_diff", (sf.col("log_Volume") - sf.lag("log_Volume", 5).over(w))) 
df = df.withColumn("biweekly_volume_diff", (sf.col("log_Volume") - sf.lag("log_Volume", 10).over(w))) 
df = df.withColumn("monthly_volume_diff", (sf.col("log_Volume") - sf.lag("log_Volume", 21).over(w))) 
df = df.withColumn("quarterly_volume_diff", (sf.col("log_Volume") - sf.lag("log_Volume", 84).over(w))) 
 
df=df.drop("ticker", "Close", "Adj_close", "Open", "High", "Low", "Volume")

d ## Removing Null values

In [16]:
df = df.na.drop()

d ## Converting to vector

In [18]:
featureassembler = VectorAssembler(inputCols=['Log_Adj_Close','log_open','log_high','log_low','log_return','weekly_log_return','biweekly_log_return',                                               'monthly_log_return','quarterly_log_return','daily_volume_diff', 'weekly_volume_diff',        
                                               'biweekly_volume_diff',   'monthly_volume_diff','quarterly_volume_diff', ], 
                                              outputCol="features")

output = featureassembler.transform(df)

df = output.select('Date', 'features','pdaily','pweekly','pbiweekly','pmonthly','pquarterly')

## Param Grid Builder
ParamGridBuilder constructs a grid of parameters to search over.
TrainValidationSplit will try all combinations of values and determine best model using the evaluator

In [20]:
lr = LinearRegression(maxIter=10)
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

## Training (80%) & Validation (20%)

In [22]:
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           trainRatio=0.8)



## Pedictions  -Daily

In [24]:
train_df_daily = df.withColumnRenamed("pdaily", "label").select("features", "label")
test_df_daily = df.withColumnRenamed("pdaily", "label").select("features", "label")



model = tvs.fit(train_df_daily)


lr_predictions = model.transform(test_df_daily)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")


In [25]:
lr_predictions = model.transform(test_df_daily)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label")

## sMAPE

In [27]:
import numpy as np

def smape(label, Prediction):
    return 100/len(label) * np.sum(2 * np.abs(Prediction - label) / (np.abs(label) + np.abs(Prediction)))

Prediction = np.array(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect())
label = np.array(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect())

print(smape(label, Prediction))



5.85813763571

##RMSE

In [29]:
def rmse(A, B):
    return np.sqrt(((A - B) ** 2).mean())
print(rmse(Prediction,label))

0.0311040923776

## PLOT

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect(), label='True Data')

ax.plot(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect(), label='Prediction')
plt.legend()
display(fig)

## Pedictions  -Weekly

In [33]:
train_df_weekly = df.withColumnRenamed("pweekly", "label").select("features", "label")
test_df_weekly = df.withColumnRenamed("pweekly", "label").select("features", "label")

model = tvs.fit(train_df_weekly)


lr_predictions = model.transform(test_df_weekly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")


In [34]:
lr_predictions = model.transform(test_df_weekly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label")

## sMAPE

In [36]:
import numpy as np

def smape(label, Prediction):
    return 100/len(label) * np.sum(2 * np.abs(Prediction - label) / (np.abs(label) + np.abs(Prediction)))

Prediction = np.array(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect())
label = np.array(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect())

print(smape(label, Prediction))

10.766224735

##RMSE

In [38]:
def rmse(A, B):
    return np.sqrt(((A - B) ** 2).mean())
print(rmse(Prediction,label))

0.0665066991295

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect(), label='True Data')

ax.plot(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect(), label='Prediction')
plt.legend()
display(fig)

## Pedictions  -Biweekly

In [41]:
train_df_biweekly = df.withColumnRenamed("pbiweekly", "label").select("features", "label")
test_df_biweekly = df.withColumnRenamed("pbiweekly", "label").select("features", "label")

model = tvs.fit(train_df_biweekly)


lr_predictions = model.transform(test_df_biweekly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")


In [42]:
lr_predictions = model.transform(test_df_biweekly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label")

## sMAPE

In [44]:
import numpy as np

def smape(label, Prediction):
    return 100/len(label) * np.sum(2 * np.abs(Prediction - label) / (np.abs(label) + np.abs(Prediction)))

Prediction = np.array(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect())
label = np.array(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect())

print(smape(label, Prediction))

13.8397002286

##RMSE

In [46]:
def rmse(A, B):
    return np.sqrt(((A - B) ** 2).mean())
print(rmse(Prediction,label))

0.0940723230887

## PLOT

In [48]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect(), label='True Data')

ax.plot(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect(), label='Prediction')
plt.legend()
display(fig)

## Pedictions  -Monthly

In [50]:
train_df_monthly = df.withColumnRenamed("pmonthly", "label").select("features", "label")
test_df_monthly = df.withColumnRenamed("pmonthly", "label").select("features", "label")

model = tvs.fit(train_df_monthly)


lr_predictions = model.transform(test_df_monthly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")


In [51]:
lr_predictions = model.transform(test_df_monthly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label")

## sMAPE

In [53]:
import numpy as np

def smape(label, Prediction):
    return 100/len(label) * np.sum(2 * np.abs(Prediction - label) / (np.abs(label) + np.abs(Prediction)))

Prediction = np.array(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect())
label = np.array(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect())

print(smape(label, Prediction))

17.8490256386

##RMSE

In [55]:
def rmse(A, B):
    return np.sqrt(((A - B) ** 2).mean())
print(rmse(Prediction,label))

0.138239907865

## PLOT

In [57]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect(), label='True Data')

ax.plot(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect(), label='Prediction')
plt.legend()
display(fig)

## Pedictions  -Quarterly

In [59]:
train_df_quarterly = df.withColumnRenamed("pquarterly", "label").select("features", "label")
test_df_quarterly = df.withColumnRenamed("pquarterly", "label").select("features", "label")

model = tvs.fit(train_df_quarterly)


lr_predictions = model.transform(test_df_quarterly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")

In [60]:
lr_predictions = model.transform(test_df_quarterly)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label")

## sMAPE

In [62]:
import numpy as np

def smape(label, Prediction):
    return 100/len(label) * np.sum(2 * np.abs(Prediction - label) / (np.abs(label) + np.abs(Prediction)))

Prediction = np.array(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect())
label = np.array(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect())

print(smape(label, Prediction))

30.6798097491

##RMSE

In [64]:
def rmse(A, B):
    return np.sqrt(((A - B) ** 2).mean())
print(rmse(Prediction,label))

0.285478549521

## PLOT

In [66]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(lr_predictions.select('label').rdd.map(lambda row : row[0]).collect(), label='True Data')

ax.plot(lr_predictions.select('prediction').rdd.map(lambda row : row[0]).collect(), label='Prediction')
plt.legend()
display(fig)

sMAPE								
     
     Model	            Feature	         Day	 Week	       Bi weekly	1 month	    Quaterly	Train/test split	Overfitting 
 
  Linear regresion 	      Adj Close 	0.627	1.076	        1.653	      2.19	      5.16	        80% / 20%	       NO

 rmse							
 
 Linear regresion	        Adj Close	0.027	0.0476	         0.0719	      0.099	      0.216	        80% / 20%	       NO

In [68]:
sMAPE								
Model	Feature	Day	Week	Bi weekly	1 month	Quaterly	Train/test split	Overfitting 
Linear regresion	Adj Close	0.627	1.076	1.653	2.19	5.16	80% / 20%	NO
rmse								
Linear regresion	Adj Close	0.027	0.0476	0.0719	0.099	0.216	80% / 20%	NO


File "<command-4421858751880259>" , line 2 
 Model	Feature	Day	Week	Bi weekly	1 month	Quaterly	Train/test split	Overfitting 
 ^ 
 SyntaxError : invalid syntax